<a href="https://colab.research.google.com/github/Rhitabrat/MAMI/blob/main/Text%20plus%20Visual%20Embeddings/text_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook makes use of both vision and text embeddings.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# !pip install -q -U tensorflow-text
# !pip install -q tf-models-official
!pip install transformers

     |████████████████████████████████| 3.4 MB 8.0 MB/s 
     |████████████████████████████████| 895 kB 70.9 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 73.9 MB/s 
     |████████████████████████████████| 3.3 MB 66.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
# import os
# import shutil

# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text
# from official.nlp import optimization  # to create AdamW optimizer

# import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
# train_data_path = "/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN/CSVs/training_original.csv"
# df = pd.read_csv(train_data_path, delimiter="\t")

# # sort the data by file_name
# def remove_file_extension(file_name):
#     return(int(file_name[:-4]))

# def sort_by_filename(df):
#     df["filename"] = df.file_name
#     df.filename = df.filename.map(remove_file_extension)
#     df = df.sort_values('filename')
#     df = df.drop('file_name', 1)
#     return df

# df = sort_by_filename(df)
# df

,misogynous,shaming,stereotype,objectification,violence,Text Transcription,filename
1033,0,0,0,0,0,Milk Milk.zip,1
2682,0,0,0,0,0,-What are you doing? -you told me to satanize ...,2
1591,0,0,0,0,0,imgflip.com ME 1254 NEW BUGS AFTER CHANGES BUG...,3
3,1,0,1,0,1,I CARE ABOUT WOMEN'S RIGHTS alamy a a a a alam...,8
829,1,0,0,0,1,"ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S...",10
...,...,...,...,...,...,...,...
256,1,0,0,1,0,YIN AND YAND In perfect armony memecenter,14195
2096,1,0,1,1,0,THIS IS A GOOD SAMDWICH MAKER memecruch.com,14205
83,1,1,1,0,0,"JUST 1 MORE REASON To never upgrade, ever Very...",14215
1463,1,0,0,1,0,POLE DANCINC You can take the girl out of the ...,14220


In [7]:
import pandas as pd
import tensorflow as tf
import numpy as np
from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
tqdm.pandas()
import html,re
import matplotlib.pyplot as plt
# from Chapter04.svm_classification import split_dataset
# from Chapter05.twitter_sentiment import clean_data, plot_model

BATCH_SIZE = 32
DATASET_SIZE = 10000
train_dataset_path = "/content/drive/MyDrive/PSU/NLP Lab/MAMI/TRAIN/CSVs/training_original.csv"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_length = 200

def split_dataset(df, train_column_name, gold_column_name, test_percent):
    X_train, X_test, y_train, y_test = train_test_split(df[train_column_name], df[gold_column_name], test_size=test_percent, stratify=df[gold_column_name], random_state=0)
    return (X_train, X_test, y_train, y_test)

def plot_model(history):
    plt.title('Loss')
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()
    plt.show()

# def segment_hashtags(tweet):
#     matches = re.findall(r'#[a-z0-9]+', tweet)
#     for match in matches:
#         tweet = re.sub(match, " ".join(segment(match)[0]), tweet)
#     return tweet

def clean_data(df):
    #Lowercase all tweets
    df['Text Transcription'] = df['Text Transcription'].progress_apply(lambda t: t.lower())
    # #Decode HTML
    # df['tweet'] = df['tweet'].progress_apply(lambda t: html.unescape(t))
    # #Remove @ mentions
    # df['tweet'] = df['tweet'].progress_apply(lambda t: re.sub(r'@[A-Za-z0-9]+','',t))
    #Remove URLs
    df['Text Transcription'] = df['Text Transcription'].progress_apply(lambda t: re.sub('https?://[A-Za-z0-9./]+','',t))
    #Segment hashtags
    # df['tweet'] = df['tweet'].progress_apply(lambda t: segment_hashtags(t))
    #Remove remaining non-alpha characters
    df['Text Transcription'] = df['Text Transcription'].progress_apply(lambda t: re.sub("[^a-zA-Z]", " ", t))
    #Re-label positive tweets with 1 instead of 4
    # df['sentiment'] = df['sentiment'].apply(lambda t: 1 if t==4 else t)
    return df

def map_inputs_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_example(input_text):
    tokenized = tokenizer.tokenize(input_text)
    bert_input = tokenizer.encode_plus(
                    input_text,                      
                    add_special_tokens = True, # add [CLS], [SEP]
                    max_length = max_length, # max length of the text that can go to BERT
                    pad_to_max_length = True, # add [PAD] tokens
                    return_attention_mask = True, # add attention mask to not focus on pad tokens
                    return_tensors='tf'
        )
    return bert_input

def encode_data(df):
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    for index, row in df.iterrows():
        tweet = row['Text Transcription']
        label = row['misogynous']
        bert_input = tokenizer.encode_plus(
                        tweet,                      
                        add_special_tokens = True, # add [CLS], [SEP]
                        max_length = max_length, # max length of the text that can go to BERT
                        pad_to_max_length = True, # add [PAD] tokens
                        return_attention_mask = True, # add attention mask to not focus on pad tokens
            )        
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_inputs_to_dict)


def prepare_dataset(df):
    df = clean_data(df)
    # df = pd.concat([df.head(size),df.tail(size)])
    # df = df.sample(frac = 1)
    ds = encode_data(df)
    return ds

def load_existing_model(export_dir):
    model = TFBertForSequenceClassification.from_pretrained(export_dir)
    return model


def get_test_train_val_datasets(ds, size=DATASET_SIZE, batch_size=BATCH_SIZE):
    ds.shuffle(32)
    train_size = int(0.7 * size)
    # val_size = int(0.15 * size)
    test_size = int(0.3 * size)
    train_dataset = ds.take(train_size).batch(batch_size)
    # test_dataset = ds.skip(train_size)
    # val_dataset = test_dataset.skip(test_size).batch(batch_size)
    val_dataset = ds.skip(train_size).batch(batch_size)
    # test_dataset = test_dataset.take(test_size).batch(batch_size)
    # print("Val size: ", val_dataset.shape)
    # print("Train size: ", train_dataset.shape)
    return (train_dataset, val_dataset)

def fine_tune_model(ds, export_dir):
    # (train_dataset, test_dataset, val_dataset) = get_test_train_val_datasets(ds)
    (train_dataset, val_dataset) = get_test_train_val_datasets(ds)
    (train_dataset, val_dataset) = get_test_train_val_datasets(ds)
    learning_rate = 2e-5
    number_of_epochs = 2
    model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
    bert_history = model.fit(train_dataset, epochs=number_of_epochs, validation_data=val_dataset)
    model.save_pretrained(export_dir)
    return model

def evaluate_model(model, X_test, y_test):
    y_pred = []
    for tweet in X_test:
        bert_input = encode_example(tweet)
        tf_output = model.predict([bert_input['input_ids'], bert_input['token_type_ids'], bert_input['attention_mask']])[0]
        tf_pred = tf.nn.softmax(tf_output, axis=1).numpy()[0]
        new_label = np.argmax(tf_pred, axis=-1)
        y_pred.append(new_label)
    
    y_pred.to_csv("/content/drive/MyDrive/PSU/NLP Lab/MAMI/answer.txt", index=False)
    print(classification_report(y_test, y_pred, labels=[0, 1], target_names=['misogynistic', 'non-misogynistic']))

def test_new_example(model_path, tweet):
    model = load_existing_model(model_path)
    bert_input = encode_example(tweet)
    tf_output = model.predict([bert_input['input_ids'], bert_input['token_type_ids'], bert_input['attention_mask']])[0]
    tf_pred = tf.nn.softmax(tf_output, axis=1).numpy()[0]
    new_label = np.argmax(tf_pred, axis=-1)
    print(new_label)
    return new_label

def remove_file_extension(file_name):
    return(int(file_name[:-4]))

def sort_by_filename(df):
    df["filename"] = df.file_name
    df.filename = df.filename.map(remove_file_extension)
    df = df.sort_values('filename')
    df = df.drop('file_name', 1)
    return df

def read_and_sort_data(path):
    df = pd.read_csv(path, sep='\t')
    return sort_by_filename(df)

def main():
    # df = pd.read_csv(dataset_path, encoding="latin1")
    # df = pd.read_csv(dataset_path)
    df = read_and_sort_data(train_dataset_path, )
    dataset = prepare_dataset(df)
    model = fine_tune_model(dataset, '/content/drive/MyDrive/PSU/NLP Lab/MAMI/Saved Models')


if(__name__ == "__main__"):
    main()
    #test_new_example('Chapter04/bert_twitter_test_model', "I hate going to school")
    #load_and_evaluate_existing_model('Chapter05/bert_twitter_test2_model')

100%|██████████| 10000/10000 [00:00<00:00, 162981.17it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
All model checkpoint layers were used when

Epoch 1/2
219/219 [==============================] - 199s 851ms/step - loss: 0.5476 - accuracy: 0.7153 - val_loss: 0.4393 - val_accuracy: 0.8007
Epoch 2/2
219/219 [==============================] - 184s 839ms/step - loss: 0.3641 - accuracy: 0.8446 - val_loss: 0.4715 - val_accuracy: 0.8100


In [23]:
# def load_and_evaluate_existing_model(export_dir, test_data_path):
#     model = load_existing_model(export_dir)
#     # df = pd.read_csv(dataset_path, encoding="latin1")
#     df = read_and_sort_data(test_data_path)
#     # df = pd.read_csv(dataset_path)
#     df = clean_data(df)

#     # (X_train, X_test, y_train, y_test) = split_dataset(df, 'Text Transcription', 'misogynous', 0.3)
#     # evaluate_model(model, X_test, y_test)
#     evaluate_model(model, df["Text Transcription"], df['misogynous'])

# test_data_path = "/content/drive/MyDrive/PSU/NLP Lab/MAMI/TEST/CSV/Test.csv"
# load_and_evaluate_existing_model('/content/drive/MyDrive/PSU/NLP Lab/MAMI/Saved Models', test_data_path)

In [ ]:
model = load_existing_model("/content/drive/MyDrive/PSU/NLP Lab/MAMI/Saved Models")
df = read_and_sort_data(test_data_path)
# df = pd.read_csv(dataset_path)
df = clean_data(df)
X_test = df["Text Transcription"]

y_pred = []
for tweet in X_test:
    bert_input = encode_example(tweet)
    tf_output = model.predict([bert_input['input_ids'], bert_input['token_type_ids'], bert_input['attention_mask']])[0]
    tf_pred = tf.nn.softmax(tf_output, axis=1).numpy()[0]
    new_label = np.argmax(tf_pred, axis=-1)
    y_pred.append(new_label)

In [21]:
# '''
# Use only for generating submission file
# '''

# test_1 = pd.read_csv(test_data_path, sep='\t')
# sorted_1 = sort_by_filename(test_1)

# sorted_1['filename'] = sorted_1['filename'].astype(str) + '.jpg'
# sorted_1 = sorted_1[['filename','Text Transcription']]
# result = pd.DataFrame(list(zip(sorted_1.filename, y_pred)))
# result

In [22]:
# result.to_csv("/content/drive/MyDrive/PSU/NLP Lab/MAMI/answer.txt", index=False, sep='\t')